In [1]:
# @TODO: try this for y-values
# sometimes this is called “non-parametric” in classical statistics
# rank-based /percentile cutoff

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np

from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
                            accuracy_score, 
                            precision_score, 
                            recall_score, 
                            f1_score, 
                            roc_curve, 
                            roc_auc_score, 
                            confusion_matrix
                            )
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix

from unbalanced_dataset import OverSampler, SMOTE

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.tsa.ar_model import AR, ARResults

%matplotlib inline

/Users/emilyhoughkovacs/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
CONFIG = {}

with open('config.txt', "r") as in_file:
    for line in in_file:
        line = line.split(":")
        parameter = line[0].strip()
        value = line[1].strip()
        CONFIG[parameter] = value
                
engine_path = 'postgresql://'+CONFIG['username']+':'+\
                CONFIG['password']+'@'+CONFIG['ip']+\
                ':'+CONFIG['port']+'/'+CONFIG['database']
        
engine = create_engine(engine_path)

##Want something like:

In [5]:
def makeSQLQuery(table_names, year, seriesnames):
    dotyyear = '.y' + str(year)
    short = table_names[0][0:2]
    seriesnames = seriesnames
    abb = [ser[0:2] for ser in seriesnames]
    query = 'SELECT x.*, y' + dotyyear + ' as zscores FROM (' + \
            'SELECT ' + short + '.countrycode AS countrycode, ' + \
            short + dotyyear + ' AS ' + table_names[0] 
    
    mystr = ''
    for s in seriesnames:
        mystr = mystr + ', env.' + s
    mystr = mystr
    
    query = query + mystr
    
    query = query + ' FROM (SELECT countrycode, ' + dotyyear[1:] + \
        ' FROM ' + table_names[0] + ') ' + short + ' JOIN ' + \
        '(SELECT ' + abb[0] + '.countrycode AS countrycode'
    
    mystr = ''
    for a, ser in zip(abb, seriesnames):
        mystr = mystr + ', ' + a + dotyyear + ' as ' + ser
    query = query + mystr
    
    query = query + ' FROM ('
    
    subquery = []
    for a, ser in zip(abb, seriesnames):
        mystr = ''
        mystr = mystr + '(SELECT countrycode, ' + dotyyear[1:] + \
        ' FROM ' + table_names[1] + " WHERE series='" + ser + \
        "') " + a
        subquery.append(mystr)
    
    query = query + subquery[0] + ' JOIN ' + subquery[1] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[1] + '.countrycode' + \
        ' JOIN ' + subquery[2] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[2] + '.countrycode' + \
        ' JOIN ' + subquery[3] + ' ON ' + \
        abb[0] + '.countrycode = ' + abb[3] + '.countrycode)) env' + \
        ' ON ' + short + '.countrycode = env.countrycode'
        
    query = query + ') x JOIN (select countrycode, ' + dotyyear[1:] + \
            ' FROM zscores) y on x.countrycode = y.countrycode'
    
    return query

In [6]:
years = [x for x in range(2006, 2015)]

In [7]:
tables = ['forestarea', 'environment']
features = ['precipitation', 'extreme', 'arable_land', 'freshwater']
print makeSQLQuery(tables, years[0], features)

SELECT x.*, y.y2006 as zscores FROM (SELECT fo.countrycode AS countrycode, fo.y2006 AS forestarea, env.precipitation, env.extreme, env.arable_land, env.freshwater FROM (SELECT countrycode, y2006 FROM forestarea) fo JOIN (SELECT pr.countrycode AS countrycode, pr.y2006 as precipitation, ex.y2006 as extreme, ar.y2006 as arable_land, fr.y2006 as freshwater FROM ((SELECT countrycode, y2006 FROM environment WHERE series='precipitation') pr JOIN (SELECT countrycode, y2006 FROM environment WHERE series='extreme') ex ON pr.countrycode = ex.countrycode JOIN (SELECT countrycode, y2006 FROM environment WHERE series='arable_land') ar ON pr.countrycode = ar.countrycode JOIN (SELECT countrycode, y2006 FROM environment WHERE series='freshwater') fr ON pr.countrycode = fr.countrycode)) env ON fo.countrycode = env.countrycode) x JOIN (select countrycode, y2006 FROM zscores) y on x.countrycode = y.countrycode


In [8]:
df = pd.DataFrame()

In [9]:
for year in years:
    query = makeSQLQuery(tables, year, features)
    year_data = pd.read_sql_query(query, engine)
    year_data['year'] = year
    year_data = year_data.set_index(['countrycode', 'year'])
    df = pd.concat([df, year_data])

In [11]:
df.head()

,,forestarea,precipitation,extreme,arable_land,freshwater,zscores
countrycode,year,,,,,,
ARG,2006,10.913914,591,0.167190,12.7358,7305.496838,-0.278010269
AUS,2006,16.499616,534,3.047099,6.21103,23622.500912,-0.279337781
BRA,2006,60.429569,1761,0.482451,8.31979,29364.390723,-0.27919335
CHN,2006,20.723562,645,7.950971,11.5784,2134.480626,-0.272709544
FRA,2006,29.172541,867,0.005718,33.4905,3124.207769,-0.279344809


In [41]:
countries = pd.Series([x for x, y in df.index])

In [39]:
years = pd.Series([y for x, y in df.index]).unique()

In [42]:
autoregressive = pd.DataFrame(index=countries, columns=years)

In [31]:
for iso, year in df.index:
    features_by_country = df.loc[iso]
    del features_by_country['zscores']
for col in features_by_country:
    print features_by_country[col].mean()

66.56870702
1020.0
4.19967674422
4.58440387952
5775.04376159


In [16]:
for countrycode, year in df.index:
    print countrycode

ARG
AUS
BRA
CHN
FRA
DEU
IND
IDN
ITA
JPN
KOR
MEX
NLD
SAU
ESP
RUS
CHE
ARM
TUR
UKR
GBR
USA
AFG
ALB
DZA
AGO
BEL
AUT
BGD
AZE
BLR
BIH
BLZ
BEN
BTN
BOL
BWA
BGR
BFA
BDI
CPV
KHM
CZE
CMR
CAN
CAF
HRV
TCD
CHL
COL
COM
ZAR
COG
CRI
CIV
CUB
CYP
DNK
DJI
ERI
DOM
ECU
EGY
SLV
EST
ETH
FJI
FIN
GMB
GHA
GRC
GEO
GTM
GIN
GNB
GUY
HTI
HND
HUN
IRN
IRQ
IRL
ISR
JAM
JOR
KAZ
KGZ
KEN
LVA
PRK
KWT
LAO
LBN
LSO
LUX
LBR
LBY
LTU
MKD
MDG
MWI
MYS
MDV
MLI
MRT
MUS
MDA
MNG
MAR
MOZ
MMR
NAM
NPL
NZL
NIC
NER
NGA
NOR
PAK
PAN
PNG
PRY
PER
PHL
POL
PRT
PRI
SVN
ROM
RWA
SVK
SRB
SEN
SLE
SLB
SOM
ZAF
LKA
SDN
SUR
TJK
SWZ
TKM
SWE
SYR
TZA
THA
TMP
TGO
TTO
TUN
UGA
URY
UZB
VEN
VNM
YEM
ZMB
ARG
AUS
BRA
CHN
FRA
DEU
IND
IDN
ITA
JPN
KOR
MEX
NLD
SAU
ESP
RUS
CHE
ARM
TUR
UKR
GBR
USA
AFG
ALB
DZA
AGO
BEL
AUT
BGD
AZE
BLR
BIH
BLZ
BEN
BTN
BOL
BWA
BGR
BFA
BDI
CPV
KHM
CZE
CMR
CAN
CAF
HRV
TCD
CHL
COL
COM
ZAR
COG
CRI
CIV
CUB
CYP
DNK
DJI
ERI
DOM
ECU
EGY
SLV
EST
ETH
FJI
FIN
GMB
GHA
GRC
GEO
GTM
GIN
GNB
GUY
HTI
HND
HUN
IRN
IRQ
IRL
ISR
JAM
JOR
KAZ
KGZ
KEN
LVA
PRK
KWT
LAO
